this looks at the clusters from time perm cluster stats and gets their average length across electrodes, and also bins them in a histogram to see the distribution  
also should get the average cluster length

In [2]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

group_mat = []

adjusted_sampling_freq = 1024

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_fixationCrossBase_1sec_mirror_0to1Test"
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Assuming `mat` is your array and `save_dir` is the directory where you want to save it
    mat_save_path = os.path.join(save_dir, f'{output_name}_mat.npy')

    # load the mat array
    mat = np.load(mat_save_path)

    group_mat.append(mat)

# Convert group_mat to a numpy array for calculation
group_mat_np = np.vstack(group_mat)  # Vertically stack the matrices

# Calculate the sum of each row for histogram
row_sums = group_mat_np.sum(axis=1)

row_sums_sig_electrodes = row_sums[row_sums > 0]

# Convert the sums to milliseconds
ms_per_sample = 1000 / adjusted_sampling_freq
row_sums_ms = row_sums * ms_per_sample
row_sums_sig_electrodes_ms = row_sums_sig_electrodes * ms_per_sample

# Plotting the histogram
plt.hist(row_sums_ms, bins='auto')  # 'auto' lets matplotlib decide the number of bins
plt.title('Histogram of Significant Timepoints per Electrode')
plt.xlabel('Total Duration of Significance (ms)')
plt.ylabel('# of Electrodes in bin')
plt.show()

KeyboardInterrupt: 

plot only sig electrodes

In [16]:
# Plotting the histogram
plt.hist(row_sums_sig_electrodes_ms, bins='auto')  # 'auto' lets matplotlib decide the number of bins
plt.title('Histogram of Significant Timepoints per Electrode')
plt.xlabel('Total Duration of Significance (ms)')
plt.ylabel('# of Sig Electrodes in bin')
plt.show()

KeyboardInterrupt: 

get avg and std of row values, to see the avg and std of total sig time across electrodes

In [18]:
avg_total_sig_time = row_sums_sig_electrodes_ms.mean()
std_total_sig_time = row_sums_sig_electrodes_ms.std()

print('avg total sig time: ', avg_total_sig_time)
print('stdev total sig time: ', std_total_sig_time)

avg total sig time:  423.0825892857143
stdev total sig time:  268.4597336795637


now get the avg length and std of each cluster, to get the avg and std of cluster size across electrodes

In [35]:
def find_clusters(row):
    """Find contiguous series of 1's and return their lengths."""
    return [len(cluster) for cluster in ''.join(map(str, row)).split('0') if cluster]

def compute_stats(group_mat_np):
    """Compute average and standard deviation of cluster lengths."""
    cluster_lengths = [find_clusters(row) for row in group_mat_np]
    avg_lengths = [np.mean(clusters) if clusters else 0 for clusters in cluster_lengths]
    std_lengths = [np.std(clusters) if clusters else 0 for clusters in cluster_lengths]
    
    # For significant electrodes only (excluding rows with all zeroes)
    sig_rows = group_mat_np[np.any(group_mat_np == 1, axis=1)]
    sig_cluster_lengths = [find_clusters(row) for row in sig_rows]
    sig_avg_lengths = [np.mean(clusters) if clusters else 0 for clusters in sig_cluster_lengths]
    sig_std_lengths = [np.std(clusters) if clusters else 0 for clusters in sig_cluster_lengths]
    
    return cluster_lengths, avg_lengths, std_lengths, sig_avg_lengths, sig_std_lengths

# Compute the statistics
cluster_lengths, avg_lengths, std_lengths, sig_avg_lengths, sig_std_lengths = compute_stats(group_mat_np) # honestly i only need sig_avg_lengths

# Convert sig_avg_lengths to a numpy array to perform element-wise multiplication
sig_avg_lengths_np = np.array(sig_avg_lengths)
sig_avg_lengths_ms = sig_avg_lengths_np * ms_per_sample

# Create histograms of average cluster lengths
# For all electrodes
# plt.hist(avg_lengths, bins=np.arange(min(avg_lengths), max(avg_lengths) + 1, 1), alpha=0.5, label='All Electrodes')
# For significant electrodes
plt.hist(sig_avg_lengths_ms, bins=np.arange(min(sig_avg_lengths_ms), max(sig_avg_lengths_ms) + 1, 1), alpha=0.5)
plt.title('Histogram of Average Cluster Lengths (ms)')
plt.xlabel('Average Cluster Length (ms)')
plt.ylabel('Number of Electrodes with this Avg Cluster Length')
plt.legend()
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


KeyboardInterrupt: 

In [27]:
cluster_lengths

[[],
 [],
 [168, 221],
 [132, 488],
 [290],
 [902],
 [1025],
 [359],
 [],
 [98],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [93],
 [],
 [91, 111],
 [105, 102, 98, 86],
 [134, 147, 239],
 [243, 676],
 [848],
 [939],
 [1013],
 [969],
 [1025],
 [937],
 [843],
 [],
 [],
 [],
 [],
 [],
 [],
 [147],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [892],
 [420, 132, 185],
 [],
 [],
 [135],
 [],
 [],
 [],
 [],
 [],
 [695],
 [937],
 [],
 [],
 [],
 [424],
 [149, 514, 115],
 [871],
 [952],
 [135, 718],
 [1025],
 [1025],
 [394, 142],
 [103, 114],
 [659],
 [826],
 [467],
 [334],
 [],
 [],
 [],
 [287],
 [460],
 [469],
 [467],
 [391],
 [215],
 [],
 [231],
 [365],
 [186, 143],
 [402],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [142],
 [342],
 [172, 363],
 [431],
 [325],
 [327],
 [131],
 [],
 [],
 [142],
 [305],
 [],
 [],
 [],
 [160],
 [129, 313],
 [],
 [],
 [],
 [],
 [113],
 [191],
 [],
 [],
 [],
 [],
 [224],
 [241],
 [],
 [],
 [],
 [],
 [139],
 [133],
 [],
 [],
 [124, 126, 96],
 [641],
 [142, 286],
 [],
 [],
 [

In [28]:
avg_lengths

[0,
 0,
 194.5,
 310.0,
 290.0,
 902.0,
 1025.0,
 359.0,
 0,
 98.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 93.0,
 0,
 101.0,
 97.75,
 173.33333333333334,
 459.5,
 848.0,
 939.0,
 1013.0,
 969.0,
 1025.0,
 937.0,
 843.0,
 0,
 0,
 0,
 0,
 0,
 0,
 147.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 892.0,
 245.66666666666666,
 0,
 0,
 135.0,
 0,
 0,
 0,
 0,
 0,
 695.0,
 937.0,
 0,
 0,
 0,
 424.0,
 259.3333333333333,
 871.0,
 952.0,
 426.5,
 1025.0,
 1025.0,
 268.0,
 108.5,
 659.0,
 826.0,
 467.0,
 334.0,
 0,
 0,
 0,
 287.0,
 460.0,
 469.0,
 467.0,
 391.0,
 215.0,
 0,
 231.0,
 365.0,
 164.5,
 402.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 142.0,
 342.0,
 267.5,
 431.0,
 325.0,
 327.0,
 131.0,
 0,
 0,
 142.0,
 305.0,
 0,
 0,
 0,
 160.0,
 221.0,
 0,
 0,
 0,
 0,
 113.0,
 191.0,
 0,
 0,
 0,
 0,
 224.0,
 241.0,
 0,
 0,
 0,
 0,
 139.0,
 133.0,
 0,
 0,
 115.33333333333333,
 641.0,
 214.0,
 0,
 0,
 137.0,
 0,
 0,
 0,
 0,
 352.0,
 954.0,
 927.0,
 921.0,
 929.0,
 918.0,
 923.0,
 345.0,
 157.0,
 180.0,
 0,
 0,
 93.0,
 214.0,

In [30]:
sig_avg_lengths

[194.5,
 310.0,
 290.0,
 902.0,
 1025.0,
 359.0,
 98.0,
 93.0,
 101.0,
 97.75,
 173.33333333333334,
 459.5,
 848.0,
 939.0,
 1013.0,
 969.0,
 1025.0,
 937.0,
 843.0,
 147.0,
 892.0,
 245.66666666666666,
 135.0,
 695.0,
 937.0,
 424.0,
 259.3333333333333,
 871.0,
 952.0,
 426.5,
 1025.0,
 1025.0,
 268.0,
 108.5,
 659.0,
 826.0,
 467.0,
 334.0,
 287.0,
 460.0,
 469.0,
 467.0,
 391.0,
 215.0,
 231.0,
 365.0,
 164.5,
 402.0,
 142.0,
 342.0,
 267.5,
 431.0,
 325.0,
 327.0,
 131.0,
 142.0,
 305.0,
 160.0,
 221.0,
 113.0,
 191.0,
 224.0,
 241.0,
 139.0,
 133.0,
 115.33333333333333,
 641.0,
 214.0,
 137.0,
 352.0,
 954.0,
 927.0,
 921.0,
 929.0,
 918.0,
 923.0,
 345.0,
 157.0,
 180.0,
 93.0,
 214.0,
 626.0,
 618.0,
 617.0,
 620.0,
 655.0,
 602.0,
 669.0,
 453.0,
 246.0,
 343.0,
 640.0,
 613.0,
 324.0,
 256.5,
 216.0,
 312.0,
 191.0,
 136.0,
 88.0,
 193.0,
 171.0,
 657.0,
 746.0,
 697.0,
 625.0,
 620.0,
 852.0,
 860.0,
 865.0,
 783.0,
 228.66666666666666,
 221.0,
 140.5,
 176.33333333333334,
 2